# PYBULLET BASICS 

Getting started with Pybullet.

Here is the Quickstart Guide : https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit?tab=t.0


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pybullet as p 
import pybullet_data 
import time
import math

## Setting up the basics of the sim:

p.connect(p.GUI) should only be called once per runtime.
In Jupyter notebooks (like Colab), that means you can’t call it in multiple cells or split your simulation setup across cells that each try to connect.

Once you’ve called p.connect(p.GUI), everything else (like loading URDFs, setting gravity, etc.) should run in the same cell or cells after it — just don’t call p.connect() again.

If you do try to reconnect, PyBullet might silently fail or open a second (invisible) connection, and the GUI won’t show anything. However if you use p.DIRECT, this issue won’t happen — but you won’t see the simulation either.


I'm repeating p.connect(p.GUI) multiple times here just to make the explanation easier to follow.

In [ ]:
p.connect(p.GUI) # getting visual of simulations 
# p.connect(p.DIRECT) # no visualizations but still doing everything so its faster
p.resetSimulation()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.8)
p.setRealTimeSimulation(0)

### Loading the desired robot:

URDF (Unified Robot Description Format) is a file format for specifying the geometry and organization of robots in ROS. 

You can choose simple or premade objects/robots from https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet/gym/pybullet_data or build more complex URDF models with the help of the turotial https://docs.ros.org/en/humble/Tutorials/Intermediate/URDF/URDF-Main.html. 

In [ ]:
p.loadURDF("plane.urdf", [0,0,0], [0,0,0,1])
targit = p.loadURDF("franka_panda/panda.urdf",[0,0,0], [0,0,0,1], useFixedBase = True ) # each urdf is basivally a set of links . # fix base for objects so it doesnt move

In [ ]:
block = p.loadURDF("block.urdf",[0,0,0], [0,0,0,1], useFixedBase = True ) # each urdf is basivally a set of links . # fix base for objects so it doesnt move

### Run the sim

In [ ]:
# Setting up the camera and starting the sim :
for step in range(300):
    focus_position , _ = p.getBasePositionAndOrientation(targit) # focus of position and orientation
    p.resetDebugVisualizerCamera(cameraDistance = 3,  # how far the camera is from the robot 
                                 cameraYaw = 0, 
                                 cameraPitch = -50, 
                                 cameraTargetPosition = focus_position) # we want to set the camera to be forcused on the focus position
    p.stepSimulation()
    time.sleep(2.01) # so that it doesnt closes very fast


### Parameters:

addUserDebugParameter lets you add custom sliders and buttons to tune parameters. It will return a unique id.

In [ ]:
angle = p.addUserDebugParameter('Steering', -0.5, 0.5, 0)
throttle = p.addUserDebugParameter('Throttle', 0, 20, 0)

### Friction
Change the friction of the ground:

In [ ]:
p.changeDynamics(
    bodyUniqueId=targit,
    linkIndex=-1,
    lateralFriction=0.5  # Typical values: 0.1 (slippery) to 10.0 (sticky)
)

### Force

Adding external force to an object:


In [ ]:
# choose what point to add the force to 
center_of_mass = p.getBasePositionAndOrientation(targit)[0]  # Choosing the center of the mass 


p.applyExternalImpulse(
    objectUniqueId=targit,
    linkIndex=-1,
    impulseObj=[5, 0, 0],
    posObj=center_of_mass, # position of the force
    flags=p.WORLD_FRAME
)

# running the sim (applying the force)
for _ in range(1000):
    p.stepSimulation()
    p.applyExternalForce(targit, -1, [0.1, 0, 0], center_of_mass, p.WORLD_FRAME)
    time.sleep(1./240.)

### 	Real-time Position/Orientation

In [ ]:
link_index = 11 
link_state = p.getLinkState(targit, link_index)
position = link_state[0]  # World position of the link's origin
orientation = link_state[1]  # World orientation (quaternion)


block_state = p.getBasePositionAndOrientation(block)
position = block_state[0]  # World position of the block's origin
orientation = block_state[1]  # World orientation (quaternion)


""" 
getBasePositionAndOrientation(body_id) : Get the position and orientation of the entire object’s base => applies to Rigid objects (e.g., block, workspace, target, robot base)
getLinkState(body_id, link_index) : Get the pose of a specific link of a robot => applies to Articulated bodies (multi-link, like robots)

"""

### Joints
A robot is a combination of links and joints. Here we get the number of the joints, join info and adding force to the joints, .. :

In [ ]:
nb_joints = p.getNumJoints(targit)
print("number of joints in this robot:", nb_joints)


for joint in range(nb_joints):
    print("joint index:",p.getJointInfo(targit, joint)[0],
          "joint type:",p.getJointInfo(targit, joint)[2],
          "joint upper bound:",p.getJointInfo(targit, joint)[9],
          "joint lower bound:",p.getJointInfo(targit, joint)[8])

# # getJointInfo: 
# # jointIndex, jointName, jointType, qIndex, uIndex, flags, jointDamping, jointFriction, jointLowerLimit, 
# # jointUpperLimit, jointMaxForce, jointMaxVelocity, linkName, jointAxis, parentFramePos, parentFrameOrn, parentIndex

number of joints in this robot: 12
joint index: 0 joint type: 0 joint upper bound: 2.9671 joint lower bound: -2.9671
joint index: 1 joint type: 0 joint upper bound: 1.8326 joint lower bound: -1.8326
joint index: 2 joint type: 0 joint upper bound: 2.9671 joint lower bound: -2.9671
joint index: 3 joint type: 0 joint upper bound: 0.0 joint lower bound: -3.1416
joint index: 4 joint type: 0 joint upper bound: 2.9671 joint lower bound: -2.9671
joint index: 5 joint type: 0 joint upper bound: 3.8223 joint lower bound: -0.0873
joint index: 6 joint type: 0 joint upper bound: 2.9671 joint lower bound: -2.9671
joint index: 7 joint type: 4 joint upper bound: -1.0 joint lower bound: 0.0
joint index: 8 joint type: 4 joint upper bound: -1.0 joint lower bound: 0.0
joint index: 9 joint type: 1 joint upper bound: 0.04 joint lower bound: 0.0
joint index: 10 joint type: 1 joint upper bound: 0.04 joint lower bound: 0.0
joint index: 11 joint type: 4 joint upper bound: -1.0 joint lower bound: 0.0
joint info f

In [ ]:
joint_id = 1
jlower = p.getJointInfo(targit, joint_id)[8]
jupper = p.getJointInfo(targit, joint_id)[9]

# to change the angles in those joints ( add force):
for step in range(2):
    j_two_target = np.random.uniform(jlower, jupper)
    j_four_target = np.random.uniform(jlower, jupper)
    p.setJointMotorControlArray(targit, [2,4], p.POSITION_CONTROL, targetPositions = [j_two_target, j_four_target] )   # takes id value, joint index, types of control , 
    p.stepSimulation()
    print("joint info for 2nd and 4th joint ", p.getJointStates(targit,[2,4])) # give the joint information for 2nd and 4th joint

#### Visualize each joint on the robot:

In [ ]:
# show the joints and joint numebrs
targit = p.loadURDF("franka_panda/panda.urdf",[0,0,0], [0,0,0,1], useFixedBase = True )
for i in range(p.getNumJoints(targit)):
    info = p.getJointInfo(targit, i)
    print(i, info[12].decode('utf-8'))
    p.addUserDebugText(str(i), [0, 0, 0.1], parentObjectUniqueId=targit, parentLinkIndex=i)
    time.sleep(4.05)

### Exploring the Effects of Varying Forces on Specific Joints:

#### Moving one joint after the other :

In [ ]:
robot_id = targit

# move joint 2
target_joint = 2
target_angle = 1.5

for _ in range(240):  # simulate ~1 sec
    p.setJointMotorControlArray(
        robot_id, target_joint, controlMode=p.POSITION_CONTROL, targetPosition=target_angle
    )
    p.stepSimulation()
    time.sleep(1 / 240.0)



# then move joint 5
target_joint = 5
target_angle = 2

for _ in range(240):  
    p.setJointMotorControlArray(
        robot_id, target_joint, controlMode=p.POSITION_CONTROL, targetPosition=target_angle
    )
    p.stepSimulation()
    time.sleep(1 / 240.0)

print("Done.")



#### Moving two joints together: 

In [ ]:
p.setJointMotorControlArray(
    bodyUniqueId=robot_id,
    jointIndices=[1, 2],
    controlMode=p.POSITION_CONTROL,
    targetPositions=[0.5, -0.8],
    forces=[100, 100],            # Control the force
    positionGains=[0.1, 0.1],     # Optional: reduce control stiffness
    velocityGains=[1.0, 1.0],     # Optional
)


### Visualize the movements of all the joints

In [ ]:
# information on the joints
movable_joints = []
for i in range(p.getNumJoints(robot_id)):
    joint_info = p.getJointInfo(robot_id, i)
    joint_type = joint_info[2]
    joint_name = joint_info[1].decode("utf-8")
    if joint_type in [p.JOINT_REVOLUTE, p.JOINT_PRISMATIC]: # only the joints that move 
        movable_joints.append(i)
        p.addUserDebugText(
            f"{i}: {joint_name}",
            [0, 0, 0.1],
            parentObjectUniqueId=robot_id,
            parentLinkIndex=i,
            textColorRGB=[1, 0, 0],
            textSize=1.0,)

# move the joints one by one
for joint_index in movable_joints:
    print(f"Moving joint {joint_index}")
    for t in range(240):  
        angle = 0.5 * math.sin(t*0.05) # using sin to have a oscillating movement 
        p.setJointMotorControl2(
            robot_id,
            joint_index,
            controlMode=p.POSITION_CONTROL,
            targetPosition=angle,
            force=100,)
        p.stepSimulation()
        time.sleep(1)


while True:
    p.stepSimulation()
    time.sleep(1)

## Moving a Joint To A Target - Inverse Kinematics

Sometimes we need to move a joint to an object. For this we need to calculate joint angles needed to place a robot's end-effector at a desired position and orientation. This can be done with IK.

In [ ]:
target_position = [0.2, 0.2, 0.1] # target position for the end-effector in world coordinates
target_orientation = p.getQuaternionFromEuler([3.14, 0, 0])  # [π, 0, 0] orientation of the gripper using Euler angles, then converts to a quaternion => Flip around x → gripper faces down
 
joint_poses = p.calculateInverseKinematics(targit, 11, target_position, target_orientation)
# This calls IK to calculate the 7 joint angles of the robot arm needed to move link 11 (the panda_hand ) to the target_pos and target_orn.\
# Reminder: joints of panda robot
# 0 panda_joint1
# 1 panda_joint2
# 2 panda_joint3
# 3 panda_joint4
# 4 panda_joint5
# 5 panda_joint6
# 6 panda_joint7
# 7 panda_joint8
# 8 panda_hand_joint
# 9 panda_finger_joint1
# 10 panda_finger_joint2
# 11 panda_grasptarget_hand
for i in range(7):  # only the 7 arm joints
    p.setJointMotorControl2(targit, i, p.POSITION_CONTROL, joint_poses[i])   # This commands each joint to move to the position from IK using position control
    #Joint indices 0 to 6 are the 7 DOF joints of the Panda arm (excluding the gripper fingers)

for _ in range(240):
    p.stepSimulation()
    time.sleep(1./240.)
